# Getting Started with OPERA DSWx Product
---

**This notebook serves as an introduction to the OPERA Dynamic Water eXtent (DSWx) product and visualizing relevant raster layers for reservoir monitoring applications. <font color='red'>Note: This notebook uses provisional products, which may differ slightly from operational products. Please refer to [DSWx product specification](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_DSWX_URS309746.pdf) for more information. </font>**

In [1]:
# Notebook dependencies
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import xarray as xr
import numpy as np
import rioxarray as rio

import hvplot.xarray
from matplotlib.colors import ListedColormap
from bokeh.models import FixedTicker
import geoviews as gv
gv.extension('bokeh')



### Product Background
---

The DSWx product products map pixel-wise surface water detections using optical or SAR imagery. This notebook focuses on relevant layers in optical DSWx products derived from the Harmonized Landsat-8 and Sentinel-2 A/B (HLS) multispectral data. 



### Metadata
---

HLS products provide surface reflectance (SR) data from the Operational Land Imager (OLI) aboard the Landsat-8 remote sensing satellite and the Multi-Spectral Instrument (MSI) aboard the Sentinel-2 A/B remote sensing satellite. HLS products are distributed over projected map coordinates aligned with the Military Grid Reference System (MGRS). Each tile covers 109.8 kilometers squared divided into 3660 rows and 3660 columns at 30 meter pixel spacing. Each tile overlaps neighbors by 4900 meters in each direction.



### Raster Layers
___

The **DSWx** product is distributed as a set of Cloud-Optimized GeoTIFF (COG) files to enable download of only particular layers of interest to a given user. All L3 DSWx layers are stored in files following GeoTIFF format specifications.

___

In [2]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DSWx-HLS_T11SQA_20230409T181446Z_20230411T155832Z_L8_30_v1.0_B01_WTR.tif'

In [3]:
data = rio.open_rasterio(LOCAL_PATH)
data

<xarray.DataArray (band: 1, y: 3660, x: 3660)> Size: 13MB
[13395600 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 29kB 7e+05 7e+05 7e+05 ... 8.097e+05 8.097e+05
  * y            (y) float64 29kB 4.1e+06 4.1e+06 4.1e+06 ... 3.99e+06 3.99e+06
    spatial_ref  int64 8B 0
Attributes: (12/48)
    ACCODE:                                                                  ...
    AEROSOL_CLASS_REMAPPING_ENABLED:                                         ...
    AEROSOL_NOT_WATER_TO_HIGH_CONF_WATER_FMASK_VALUES:                       ...
    AEROSOL_PARTIAL_SURFACE_AGGRESSIVE_TO_HIGH_CONF_WATER_FMASK_VALUES:      ...
    AEROSOL_PARTIAL_SURFACE_WATER_CONSERVATIVE_TO_HIGH_CONF_WATER_FMASK_VALUE...
    AEROSOL_WATER_MODERATE_CONF_TO_HIGH_CONF_WATER_FMASK_VALUES:             ...
    ...                                                                          ...
    WORLDCOVER_SOURCE:                                                       ...
    AREA_OR_POINT:                                                           ...
    _FillValue:                                                              ...
    scale_factor:                                                            ...
    add_offset:                                                              ...
    long_name:                                                               ...

In [4]:
crs = data.rio.crs
crs

CRS.from_epsg(32611)

In [5]:
dswx = data.rename({'x':'longitude', 'y':'latitude', 'band':'band'}).squeeze()
dswx

<xarray.DataArray (latitude: 3660, longitude: 3660)> Size: 13MB
[13395600 values with dtype=uint8]
Coordinates:
    band         int64 8B 1
  * longitude    (longitude) float64 29kB 7e+05 7e+05 ... 8.097e+05 8.097e+05
  * latitude     (latitude) float64 29kB 4.1e+06 4.1e+06 ... 3.99e+06 3.99e+06
    spatial_ref  int64 8B 0
Attributes: (12/48)
    ACCODE:                                                                  ...
    AEROSOL_CLASS_REMAPPING_ENABLED:                                         ...
    AEROSOL_NOT_WATER_TO_HIGH_CONF_WATER_FMASK_VALUES:                       ...
    AEROSOL_PARTIAL_SURFACE_AGGRESSIVE_TO_HIGH_CONF_WATER_FMASK_VALUES:      ...
    AEROSOL_PARTIAL_SURFACE_WATER_CONSERVATIVE_TO_HIGH_CONF_WATER_FMASK_VALUE...
    AEROSOL_WATER_MODERATE_CONF_TO_HIGH_CONF_WATER_FMASK_VALUES:             ...
    ...                                                                          ...
    WORLDCOVER_SOURCE:                                                       ...
    AREA_OR_POINT:                                                           ...
    _FillValue:                                                              ...
    scale_factor:                                                            ...
    add_offset:                                                              ...
    long_name:                                                               ...

## **Band 1: Water classification (WTR)**
***


**Data Type:**  UInt8 <br>
**Description:** Masked interpreted water classification layer. This represents pixel-wise classification into one of three water
classes (not water, open water, and partial surface water), cloud/cloud shadow class, or no data classes.


**Layer Values:**<br> 
* **0:** Not Water – an area with valid reflectance data that is not open water (class 1), partial surface water (class 2), or
cloud/cloud shadow (class 9). Masking can result in “not water” (class 0) where land cover masking is applied<br>
* **1:** Open Water – an area that is entirely water and unobstructed to the sensor, including obstructions by vegetation, terrain,
and buildings <br>
* **2:** Partial Surface Water – an area that is at least 50% and less than 100% open water. This may be referred to as “subpixel
inundation” when referring to a pixel’s area. Examples include inundated sinkholes, floating vegetation, and pixels bisected by
coastlines <br> 
* **8:** Snow/Ice - an area identified as snow/ice according to input HLS Fmask quality assurance (QA) data <br>
* **9:** Cloud/Cloud Shadow – an area identified as cloud, cloud shadow, or snow/ice according to input quality assurance (QA)
data <br>
* **255:** Fill value (no data)  <br> 

In [6]:
# Defines colormap for visualization
levels = [0, 0.9, 1.9, 2.9, 7.9, 8.9, 10]
color_key = {
    "Not Water": "#ffffff",
    "Open Water": "#0000ff",
    "Partial Surface Water": "#00ff00",
    "Reserved": "#000000",
    "Snow/Ice": "#00ffff",
    "Clouds/Cloud Shadow": "#7f7f7f"
}

ticks = [0.5, 1.5, 2.5, 5.5, 8.5, 9.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

In [7]:
# Creates basemap
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000, padding=0.1)

In [8]:
dswx.hvplot.image(x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=True, 
                          dynamic=True, 
                          aspect='equal', 
                          frame_width=500, 
                          frame_height=500,  
                          clim=(0,10), alpha=0.8).opts(title=f"B01_WTR",color_levels=levels,cmap=tuple(color_key.values()),
                                    colorbar_opts={'ticker':ticker,'major_label_overrides':labels}) * base

:DynamicMap   []
   :Overlay
      .Image.I :Image   [longitude,latitude]   (Water classification (WTR))
      .WMTS.I  :WMTS   [Longitude,Latitude]